# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 讀取資料

In [2]:
train_X, train_y = SVHN2.traindata(Float32)
test_X,  test_y  = SVHN2.testdata(Float32)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [3]:
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×26032 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  1  0  1  0  1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  1  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  1  0  0  0  0  0  0  0  0  0  0

## 資料切分為 minibatch

In [4]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

DataLoader((Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627

## CNN 模型

In [35]:
model = Chain(
    Conv((3, 3), 3 => 32, pad = (1, 1), relu),
    MaxPool((2, 2)),
    Conv((3, 3), 32 => 32, pad = (1, 1), relu),
    MaxPool((2, 2)),
    Conv((3, 3), 32 => 64, pad = (1, 1), relu),
    MaxPool((2, 2)),
    flatten,
    Dense(1024, 256, relu),
    Dense(256, 10),
    softmax) # write your model here

Chain(Conv((3, 3), 3=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(1024, 256, relu), Dense(256, 10), softmax)

## 使用CUDA

In [36]:
using CuArrays
model = model |> gpu
train_X = train_X |> gpu
train_y = train_y |> gpu
test_X = test_X |> gpu
test_y =test_y |> gpu

10×26032 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  1  0  1  0  1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  1  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  1  0  0  0  0  0  0  0  0  0  0

## 損失函數

In [37]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

## Callback 函式

In [38]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/ length(test)
end

test_loss (generic function with 1 method)

In [39]:
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

## 模型訓練

In [40]:
epochs = 20
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.01), cb = throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\ocean_chou\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2649012f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0


┌ Info: Epoch 2
└ @ Main C:\Users\ocean_chou\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0


┌ Info: Epoch 3
└ @ Main C:\Users\ocean_chou\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0


┌ Info: Epoch 19
└ @ Main C:\Users\ocean_chou\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0


┌ Info: Epoch 20
└ @ Main C:\Users\ocean_chou\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0


## 模型評估

In [41]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [42]:
accuracy(test_X, test_y)

0.1958743085433313

## 使用已預訓練好的模型